# Imports and setup

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
pd.options.display.float_format = '{:,.2f}'.format
%matplotlib inline

# Variables

In [ ]:
subtotals_dict = {}
base_yrs = range(2017,2020)
forecast_yrs = range(2020,2022)
all_yrs = range(base_yrs[0], forecast_yrs[-1]+1)
df_common_size = pd.DataFrame(index = all_yrs)
df_growth = pd.DataFrame(index = all_yrs)

# Functions

In [ ]:
def cleanup():
    ''' This function imports data from csv file 
    and returns a dataframe object with 
    income statement data.'''
    
    # Read csv into dataframe
    df = pd.read_csv('financial_data.csv')
    
    # Change strings to lowercase
    df.columns = df.columns.str.lower()
    df['item'] = df['item'].str.lower()
    df['statement'] = df['statement'].str.lower()
    
    # Filter income statement data and drop column
    df = df[df['statement']=='income']
    df = df.drop(['statement'], axis=1)
    
    # Set income statement items as index and
    # transpose dataframe
    df = df.set_index('item')
    df = df.T
    
    # Set years in index as ints
    df.index = df.index.astype(int)
    
    return df

In [ ]:
# Calculate subtotals in income statement
def calc():
    ''' This function calculates the value
    of each subtotal column in the dataframe df.'''
    
    df['net revenues'] = df['revenue'] - df['cost of goods sold']
    df['ebitda'] = (df['net revenues']-
                    df['cost of goods sold']-
                    df['salaries and benefits']-
                    df['rent and overhead'])
    df['ebit'] = df['ebitda'] - df['interest']
    df['earnings before taxes'] = df['ebit'] - df['interest']
    df['earnings after taxes'] = df['earnings before taxes'] - df['taxes']

In [ ]:
def forecast_rev(growth_perc):
    ''' This function is used to forecast the
    values in the relevant column.'''
    
    for yr in forecast_yrs:
        df.loc[yr,'revenue'] = df.loc[yr-1,'revenue']*(1 + growth_perc)
    calc()

In [ ]:
def forecast_cogs(cogs_perc_rev):
    ''' This function is used to forecast the
    values in the relevant column.'''
    
    for yr in forecast_yrs:
        df.loc[yr, 'cost of goods sold'] = cogs_perc_rev * df.loc[yr, 'revenue']
    calc()

In [ ]:
def forecast_salaries(sals_perc_rev):
    ''' This function is used to forecast the
    values in the relevant column.'''
    
    for yr in forecast_yrs:
        df.loc[yr, 'salaries and benefits'] = sals_perc_rev * df.loc[yr, 'revenue']
    calc()

In [ ]:
def forecast_rent_overhead(rent_overhead):
    ''' This function is used to forecast the
    values in the relevant column.'''
    
    for yr in forecast_yrs:
        df.loc[yr, 'rent and overhead'] = rent_overhead
    calc()

In [ ]:
def forecast_depreciation(depr):
    ''' This function is used to forecast the
    values in the relevant column.'''
    
    for yr in forecast_yrs:
        df.loc[yr, 'depreciation & amortization'] = depr
    calc()

In [ ]:
def forecast_interest(interest):
    ''' This function is used to forecast the
    values in the relevant column.'''
    
    for yr in forecast_yrs:
        df.loc[yr, 'interest'] = interest
    calc()

In [ ]:
def forecast_taxes(tax_perc):
    ''' This function is used to forecast the
    values in the relevant column.'''
    
    for yr in forecast_yrs:
        df.loc[yr, 'taxes'] = tax_perc * df.loc[yr, 'earnings before taxes']
    calc()

In [ ]:
def subtotals(num_row, name_row):
    ''' This function is used to insert
    colums in the DataFrame df - the columns
    are displayed as rows in the transposed
    DataFrame.'''
    
    df.insert(num_row, name_row, np.nan)
    subtotals_dict[name_row] = num_row

In [ ]:
def common_size():
    ''' This function is used to create a
    DataFrame where all columns are a % of
    revenues.'''
    
    df_common_size['revenue'] = 1
    for col in df.columns[1:]:
        df_common_size[col] = df[col]/df['revenue']
    display(df_common_size.T)

In [ ]:
def growth():
    ''' This function is used to create a
    DataFrame where each element in a column
    is expressed as the % growth of the previous
    element in the same column.'''
    
    for col in df.columns:
        df_growth[col] = df[col].pct_change()
    display(df_growth.T)

In [ ]:
def plot(df):
    ''' This function is used to plot the specified
    columns in the DataFrame df as a bar chart.'''
    
    plt.bar(df.index+.17,df['earnings after taxes'], width=0.35, label='earnings after tax')
    plt.bar(df.index-.17,df['revenue'], width=0.35, label='revenue')
    plt.legend()
    

In [ ]:
def bold(x):
    '''This function returns lists which are
    used in the format_table function in order to
    format the DataFrame df.'''
    
    bold_rows = [2, 5, 7, 9, 11]
    format_list = []
    counter = 0
    
    for counter, v in enumerate(x):
        if counter in bold_rows:
            format_list.append('font-weight: bold') 
        else:
            format_list.append('')
        counter += 1
    return format_list

In [ ]:
def format_table():
    '''This function applies formatting to the
    DataFrame df and returns the DataFrame.'''
    
    return df.T.style.apply(bold).format("{:,.0f}")

In [ ]:
def f(rev_growth_perc,
      cogs_perc_rev,
      sals_perc_rev,
      rent_overhead,
      depr,
      interest,
      tax_perc):
    ''' This function is used as to call all relevant functions
    in order to fill in all of the forecast figures, calculate
    subtotals, display DataFrames and plot relevant items.'''
    
    forecast_rev(rev_growth_perc)
    forecast_cogs(cogs_perc_rev)
    forecast_salaries(sals_perc_rev)
    forecast_rent_overhead(rent_overhead)
    forecast_depreciation(depr)
    forecast_interest(interest)
    forecast_taxes(tax_perc)
    calc()
    display(df.T)
    growth()
    common_size()
    plot(df)

# Create income statement

In [ ]:
# Call cleanup function and create initial
# DataFrame
df = cleanup()
df

In [ ]:
# Call subtotals function to create columns
# with subtotals for income statement.
subtotals(2, 'net revenues')
subtotals(5, 'ebitda')
subtotals(7, 'ebit')
subtotals(9, 'earnings before taxes')
subtotals(11, 'earnings after taxes')

# Display transposed view of DataFrame
df.T

In [ ]:
# Call calc funtion to calculate values of
# subtotal columns in DataFrame df.
calc()
df.T

# Assumptions and scenario widgets

In [ ]:
# Pass assumptions to 'interactive' function from ipywidgets library
# to generate widgets which can be used to manually control
# assumptions in 'forecast' functions.
w = interactive(f,
                rev_growth_perc = 0.2,
                cogs_perc_rev = 0.1,
                sals_perc_rev = 0.2,
                rent_overhead = 75,
                depr = 100,
                interest = 20,
                tax_perc = 0.2)

In [ ]:
display(w)